In [17]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, advanced_activations
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from __future__ import print_function
import keras
from keras.datasets import cifar10

import os
import pickle
import numpy as np

import os
import pickle
import numpy as np
import pandas as pd

In [2]:
import os
os.getcwd()

'/gpfs/global_fs01/sym_shared/YPProdSpark/user/sde1-bebf9d65efdfe8-478becfe92a9/notebook/work'

In [19]:
def trainmodel(batch_size,num_classes,epochs,num_predictions):
    batch_size = batch_size
    num_classes = num_classes
    epochs = epochs
    data_augmentation = True
    num_predictions = num_predictions
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = 'keras_cifar10_trained_model.h5'

    # The data, shuffled and split between train and test sets:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
    model.add(advanced_activations.LeakyReLU(alpha=0.3))
    model.add(Conv2D(32, (3, 3)))
    model.add(advanced_activations.LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(advanced_activations.LeakyReLU(alpha=0.3))
    model.add(Conv2D(64, (3, 3)))
    model.add(advanced_activations.LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(advanced_activations.LeakyReLU(alpha=0.3))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Let's train the model using RMSprop
    model_3.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4)

    # Load label names to use in prediction results
    label_list_path = 'datasets/cifar-10-batches-py/batches.meta'


    keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
    datadir_base = os.path.expanduser(keras_dir)
    if not os.access(datadir_base, os.W_OK):
        datadir_base = os.path.join('/tmp', '.keras')
    label_list_path = os.path.join(datadir_base, label_list_path)

    with open(label_list_path, mode='rb') as f:
        labels = pickle.load(f)

    # Evaluate model with test data set and share sample prediction results
    evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                       batch_size=batch_size,
                                                       shuffle=False),
                                          steps=x_test.shape[0] // batch_size,
                                          workers=4)
    print('Model Accuracy = %.2f' % (evaluation[1]))

    predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                       batch_size=batch_size,
                                                       shuffle=False),
                                          steps=x_test.shape[0] // batch_size,
                                          workers=4)

    for predict_index, predicted_y in enumerate(predict_gen):
        actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
        predicted_label = labels['label_names'][np.argmax(predicted_y)]
        print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                              predicted_label))
        if predict_index == num_predictions:
            break


In [20]:
def createmodel(df):
    for index, row in df.iterrows():
        trainmodel(row['batch_size'],row['num_classes'],row['epochs'],row['num_predictions'])

In [18]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_f2b31d3aa8f74da9a868a4102976b52e(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_eaa7a3ccbee220b104765fd6ecbda6648009a26f','domain': {'id': 'a76f4599636a4d9fb2a2209d58f8cb53'},
            'password': 'f=O00,1/BLCkVq-Q'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

df = pd.read_csv(get_object_storage_file_with_credentials_f2b31d3aa8f74da9a868a4102976b52e('DefaultProjectlekhrajanijhuskyneuedu', 'Input.csv'))
df.head()


,batch_size,num_classes,epochs,num_predictions
0,20,10,25,20
1,32,10,30,20
2,32,10,25,20


In [ ]:
createmodel(df)